---
## Health, Wealth of Nations from 1800-2008

In [1]:
import os
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
from bqplot import Figure, Tooltip, Label
from bqplot import Axis, ColorAxis
from bqplot import LogScale, LinearScale, OrdinalColorScale
from bqplot import Scatter, Lines
from bqplot import CATEGORY10

In [3]:
from ipywidgets import HBox, VBox, IntSlider, Play, jslink

In [4]:
from more_itertools import flatten

---
### Get Data

In [5]:
year_start = 1800

In [6]:
df = pd.read_json("data_files/nations.json")

In [7]:
list_rows_to_drop = df['income'].apply(len).where(lambda i: i < 10).dropna().index.tolist()
df.drop(list_rows_to_drop, inplace=True)

In [8]:
dict_dfs = {}
for COL in ['income', 'lifeExpectancy', 'population']:
    df1 = \
    DataFrame(df
              .loc[:, COL]
              .map(lambda l: (DataFrame(l)
                              .set_index(0)
                              .squeeze()
                              .reindex(range(1800, 2009))
                              .interpolate()
                              .to_dict()))
              .tolist())
    df1.index = df.name
    dict_dfs[COL] = df1

In [9]:
def get_data(year):
    """
    """
    income = dict_dfs['income'].loc[:, year]
    lifeExpectancy = dict_dfs['lifeExpectancy'].loc[:, year]
    population = dict_dfs['population'].loc[:, year]
    return income, lifeExpectancy, population

get_min_max_from_df = lambda df: (df.min().min(), df.max().max())

---
### Create Tooltip

In [10]:
tt = Tooltip(fields=['name', 'x', 'y'], 
             labels=['Country', 'IncomePerCapita', 'LifeExpectancy'])

---
### Create Scales

In [11]:
income_min, income_max = get_min_max_from_df(dict_dfs['income'])
x_sc = LogScale(min=income_min, 
                max=income_max)

life_exp_min, life_exp_max = get_min_max_from_df(dict_dfs['lifeExpectancy'])
y_sc = LinearScale(min=life_exp_min, 
                   max=life_exp_max)

pop_min, pop_max = get_min_max_from_df(dict_dfs['population'])
size_sc = LinearScale(min=pop_min, 
                      max=pop_max)

c_sc = OrdinalColorScale(domain=df['region'].unique().tolist(), 
                         colors=CATEGORY10[:6])

---
### Create Axes

In [12]:
ax_y = Axis(label='Life Expectancy', 
            scale=y_sc, 
            orientation='vertical', 
            side='left', 
            grid_lines='solid')

ax_x = Axis(label='Income per Capita', 
            scale=x_sc, 
            grid_lines='solid')

---
## Create Marks

### 1. Scatter

In [13]:
cap_income, life_exp, pop = get_data(year_start)

In [14]:
scatter_ = Scatter(x=cap_income, 
                   y=life_exp, 
                   color=df['region'], 
                   size=pop,
                   names=df['name'], 
                   display_names=False,
                   scales={
                       'x': x_sc, 
                       'y': y_sc, 
                       'color': c_sc, 
                       'size': size_sc
                   },
                   default_size=4112, 
                   tooltip=tt, 
                   animate=True, 
                   stroke='Black',
                   unhovered_style={'opacity': 0.5})

### 2. Line

In [15]:
line_ = Lines(x=dict_dfs['income'].loc['Angola'].values, 
              y=dict_dfs['lifeExpectancy'].loc['Angola'].values, 
              colors=['Gray'],
              scales={
                  'x': x_sc, 
                  'y': y_sc
              }, 
              visible=False)

---
### Create Label

In [16]:
year_label = Label(x=[0.75], y=[0.10],
                   font_size=50, font_weight='bolder', 
                   colors=['orange'],
                   text=[str(year_start)],
                   enable_move=True)

---
## Construct the Figure

In [17]:
time_interval = 10

fig_ = \
Figure(
    marks=[scatter_, line_, year_label], 
    axes=[ax_x, ax_y],
    title='Health and Wealth of Nations', 
    animation_duration=time_interval
)

fig_.layout.min_width = '960px'
fig_.layout.min_height = '640px'

fig_

Figure(animation_duration=10, axes=[Axis(label='Income per Capita', scale=LogScale(max=119849.29, min=281.91))…

---
## Add Interactivity

- Update chart when year changes

In [18]:
slider_ = IntSlider(min=year_start, max=2008, step=1, description='Year: ', value=year_start)

def on_change_year(change):
    """
    """
    scatter_.x, scatter_.y, scatter_.size = get_data(slider_.value)
    year_label.text = [str(slider_.value)]

slider_.observe(on_change_year, 'value')

slider_

IntSlider(value=1800, description='Year: ', max=2008, min=1800)

- Display line when hovered

In [19]:
def on_hover(change):
    """
    """
    if change.new is not None:
        display(change.new)
        line_.x = dict_dfs['income'].iloc[change.new + 1]
        line_.y = dict_dfs['lifeExpectancy'].iloc[change.new + 1]
        line_.visible = True
    else:
        line_.visible = False

In [20]:
scatter_.observe(on_hover, 'hovered_point')

---
## Add Animation!

In [21]:
play_button = Play(min=1800, max=2008, interval=time_interval)
jslink((play_button, 'value'), (slider_, 'value'))

---
## Create the GUI

In [22]:
VBox([play_button, slider_, fig_])